<a href="https://colab.research.google.com/github/ValentynaGrach/ml-homeworks/blob/main/%D0%94%D0%97__16_Transformer_Translation_ValentynaGrach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧩 Крок 1: Встановлення бібліотек

- Встановимо `transformers` (моделі) та `gradio` (інтерфейс).

In [ ]:
!pip install transformers sentencepiece gradio --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import gradio as gr
import ipywidgets as widgets
from IPython.display import display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00


#📥 **КРОК 2: Завантаження моделі перекладу**

- Виберемо компактну модель, яка підтримує:  
 **українську → англійську**,     
 **українську → польську**.

- Завантажимо її з HuggingFace.

In [ ]:
# 🇬🇧 NLLB для англійської
tokenizer_en = AutoTokenizer.from_pretrained("facebook/nllb-200-1.3B")
model_en = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-1.3B")
print("🇬🇧 NLLB loaded")

# 🇵🇱 MBART для польської
tokenizer_pl = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model_pl = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
print("🇵🇱 MBART loaded")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

🇵🇱 MBART loaded


🔄 Моделі успішно завантажена!

# 🔁 **КРОК 3: Функція перекладу**

- Напишемо просту функцію `translate(text, target_lang)`, яка:
  - приймає текст українською,
  - повертає переклад англійською, норвезькою, польською.

✅ Функція перекладу 🇺🇦 → 🇬🇧 (Англійська) через mbart50

In [ ]:
def translate_uk_to_en_nllb(text):
    prompt = ">>eng_Latn<< >>ukr_Cyrl<< " + text
    encoded = tokenizer_en(prompt, return_tensors="pt", padding=True, truncation=True)
    generated = model_en.generate(**encoded)
    return tokenizer_en.decode(generated[0], skip_special_tokens=True)

✅ Функція перекладу 🇺🇦 → 🇵🇱 (Польська) через mbart50

In [ ]:
def translate_uk_to_pl_mbart50(text):
    tokenizer_pl.src_lang = "uk_UA"
    encoded = tokenizer_pl(text, return_tensors="pt", padding=True, truncation=True)
    generated = model_pl.generate(
        **encoded,
        forced_bos_token_id=tokenizer_pl.lang_code_to_id["pl_PL"])
    return tokenizer_pl.decode(generated[0], skip_special_tokens=True)

In [ ]:
# Вхідний текст українською
ukr_text = "Штучний інтелект вже займає крісла федеральних службовців США."

print("🇬🇧 EN (nllb-1.3B):", translate_uk_to_en_nllb(ukr_text))
print("🇵🇱 PL (mbart50):", translate_uk_to_pl_mbart50(ukr_text))

🇬🇧 EN (nllb-1.3B): AI is already occupying the chairs of US federal employees.
🇵🇱 PL (mbart50): Artificial intelligence jest już na krześle U.S. public servants.


✅ **Підсумок перекладача (UK → EN / PL)**
🔍 **Аналіз моделей:**
- 🇬🇧 **Англійська (`nllb-200-1.3B`)**:  
  Переклад точний, звучить природно, граматично правильно — **ідеально підходить** для англійської.

- 🇵🇱 **Польська (`mbart50`)**:  
  Переклад зрозумілий, але містить **домішки англійських слів** — можливо через близькість лексики.  
  Результат прийнятний для базового використання.

---
📌 **Висновок**:  
🔸 Обрані моделі дають **найкращий доступний баланс** якості й швидкості.  
🔸 Перекладач готовий до виводу в інтерфейс з двома стабільними мовами: **англійська та польська**.

---

# 💻 **КРОК 4: Інтерфейс**

- Створимо інтерфейс на `Gradio`:
  - поле для введення українського тексту,
  - випадаючий список мов (англійська, норвезька, польська),
  - кнопка для перекладу,
  - поле з результатом.

In [ ]:
def translate_text(text, target_lang):
    if target_lang == "Англійська":
        return translate_uk_to_en_nllb(text)
    elif target_lang == "Польська":
        return translate_uk_to_pl_mbart50(text)
    else:
        return "❌ Оберіть мову перекладу."

interface = gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(label="Введіть текст українською:"),
        gr.Radio(choices=["Англійська", "Польська"], label="Мова перекладу")],
    outputs=gr.Textbox(label="Переклад"),
    title="Перекладач української мови 🌍",
    description="Переклад з української на англійську (NLLB) або польську (MBART).")

#🧪 **КРОК 5: Тестування**

- Протестуємо переклад — введемо кілька речень українською.
- Перевіримо якість перекладу.

In [ ]:
button = widgets.Button(description="▶️ Запустити перекладач")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        print("🔄 Запуск Gradio...")
        interface.launch(share=True)

button.on_click(on_button_clicked)

display(button, output)

Button(description='▶️ Запустити перекладач', style=ButtonStyle())

Output()

✅ **Що вийшло добре:**

| Компонент | Оцінка | Коментар |
|-----------|--------|----------|
| 🇺🇸 **Стиль** | ⭐️⭐️⭐️⭐️⭐️ | Звучить як справжній новинний переклад |
| 🧠 **Зміст** | ⭐️⭐️⭐️⭐️ | Збережено повністю |
| 📌 **Гумор/іронія** | ⭐️⭐️⭐️ | Частково втрачено ("вигадування відмовок" → "making rejections") |
| 🧩 **Складні іменовані конструкції** | ⭐️⭐️⭐️⭐️ | Імена, абревіатури збережено чудово |

---
⚠️ Маленькі зауваження:

- 🟡 "making rejections" звучить незвично (можна було б: *crafting excuses* або *inventing justifications*)
- 🟢 Але для автоматичного перекладу — **рівень дуже високий!**

---
✅ Висновок:

🔹 **Модель NLLB-200-1.3B прекрасно справилась з англійською версією**.  
🔹 Стиль, структура, терміни — все звучить професійно й природно.  
🔸 Легкий гумор/іронія передано частково, але це очікувано для нейромережі.

---

# 📦 **КРОК 6: Оптимізація (опціонально)**

✅ Оптимізація: план

| Крок | Що робимо | Навіщо |
|------|-----------|--------|
| 1️⃣ | Обробка помилок | Щоб не падало, якщо поле порожнє |
| 2️⃣ | Спрощення інтерфейсу | Зручно, чисто, інтуїтивно |
| 3️⃣ | Додавання статусу | Повідомлення: "Перекладаємо..." + очищення поля

✅ 1️⃣ Обробка помилок і порожніх запитів

In [ ]:
def translate_text_export(text, target_lang):
    text = text.strip()

    if not text:
        return "⚠️ Введіть текст для перекладу."

    if target_lang == "Англійська":
        translation = translate_uk_to_en_nllb(text)
    elif target_lang == "Польська":
        translation = translate_uk_to_pl_mbart50(text)
    else:
        return "❌ Оберіть мову перекладу."

    formatted_output = (
        "🔹 Вхідний текст (українською):\n\n"
        + text + "\n\n"
        + "🔸 Переклад " + ("англійською" if target_lang == "Англійська" else "польською") + ":\n\n"
        + translation)

    return formatted_output

✅ 2️⃣ Оновлений інтерфейс — простий і чистий

In [ ]:
interface = gr.Interface(
    fn=translate_text_export,
    inputs=[
        gr.Textbox(label="Введіть текст українською:", lines=6, placeholder="Вставте або введіть свій текст тут..."),
        gr.Radio(choices=["Англійська", "Польська"], label="Мова перекладу")
    ],
    outputs=gr.Textbox(label="Результат перекладу", show_copy_button=True),
    title="🔄 Перекладач з української мови 🌐",
    description="Переклад з української на англійську (NLLB) або польську (MBART). Підтримується форматований результат із кнопкою копіювання.")

✅ 3️⃣ Кнопка запуску в стилі Google Colab

In [ ]:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="▶️ Запустити перекладач")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        print("🔄 Запуск Gradio...")
        interface.launch(share=True)

button.on_click(on_button_clicked)

display(button, output)

Button(description='▶️ Запустити перекладач', style=ButtonStyle())

Output()

# ✅ **КРОК 7: Висновки**

✅ **Фінальний висновок** - **тестування моделей:**
📤 Вхідний текст: українська публіцистика з елементами гумору, стилістично насичена.

- ✅ **NLLB-200-1.3B (facebook)**  
  🔸 показала **відмінну якість перекладу на англійську**, навіть на складних текстах з іронією та багатокомпонентною структурою.

- ⚠️ **MBART50 (facebook)**  
  🔸 показала **середній результат для польської мови**, але не змогла впоратись із довгими або стилістично насиченими реченнями.

---

Модель **MBART50 не підходить для перекладу на польську мову** у складних текстах.  
Модель **NLLB-1.3B показує високу якість перекладу на англійську**, навіть у випадках з іронією та довгими реченнями.

---

In [1]:
if 'metadata' in globals():
    for cell in notebook['cells']:
        if 'metadata' in cell and 'widgets' in cell['metadata']:
            del cell['metadata']['widgets']